In [28]:
#!pip install fasttext
#!pip install transformers
#import nltk
#nltk.download('punkt')
#!pip3 install torch torchvision torchaudio

In [ ]:
# download word embeddings file (Pre-trained Vectors) for GloVe
import os
import urllib.request
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.840B.300d.zip','glove.840B.300d.zip')
!unzip "glove.840B.300d.zip" -d "./data/"

# or directly download 'glove.840B.300d.zip' from https://nlp.stanford.edu/projects/glove/ and move to the 'data' folder
# note: this might take about 8 minutes

Archive:  glove.840B.300d.zip
  inflating: ./data/glove.840B.300d.txt  


In [29]:
DATA_DIR = "./data/"
VALID_SIZE = .2
MODEL_PATH = "model_task2.bin"

In [30]:
import pandas as pd
import numpy as np
import scipy
import nltk
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch.optim import AdamW
from fasttext import load_model
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, model_selection
from torch import LongTensor
from sklearn.metrics import f1_score, confusion_matrix, classification_report

---

In [31]:
df = pd.read_csv('../Dataset/data/restaurants_laptop_train_with_pos_task2_cleaned.csv')

In [32]:
df.polarity = df.polarity.replace(-1,2)

encoder = preprocessing.LabelEncoder()
df.loc[:, "aspect_tag"] = encoder.fit_transform(df["aspect_tag"])

sentences = df.groupby("num")["text"].apply(list).values
aspect_tags = df.groupby("num")["aspect_tag"].apply(list).values
polarity_tags = df.groupby("num")["polarity"].apply(list).values

polarity_unique_values = df.polarity.unique()

print('num of aspect tags: {}'.format(len(encoder.classes_)))
print('num of polarity tags: {}'.format(len(polarity_unique_values)))

np.where(encoder.classes_ == "AT")[0].item()

num of aspect tags: 2
num of polarity tags: 4


0

In [33]:
idx = 0
print(sentences[idx])
print(aspect_tags[idx])
print(polarity_tags[idx])

['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


---

In [34]:
def get_new_aspect_cluster(left, right, aspect_term, polarity, sentence, window_size=5):
    left = left[-window_size:]
    right = right[:window_size]

    if len(polarity) == 0:
        polarity = 0
    else:
        polarity = int(sum(polarity)/len(polarity))
   
    left.extend(aspect_term)
    left.extend(right)
    return {
        "local_context":left,
        "global_context":sentence,
        "aspect_term":aspect_term,
        "polarity":polarity,
    }

def chop(sentence, aspect_tag, polarity_tag, window_size=5):
    ret_aspect_clusters = []
#     ret_aspect_clusters = {
#         "context":list(),
#         "aspect_term":list(),
#         "polarity":list(),
#     }
    left = []
    right = []
    aspect_term = []
    polarity = []
    doing_left = True
    doing_right = False
    doing_aspect = False
    for i in range(len(sentence)):
        # check if the current token is an aspect term
        if aspect_tag[i] == 0:
            if doing_left:
                doing_aspect = True
                doing_left = False
            elif doing_right:
                doing_right = False
                doing_aspect = True
                # Now, need to save the previous aspect term cluster
                ret_aspect_clusters.append(get_new_aspect_cluster(
                    left, right, aspect_term, polarity, sentence, window_size=window_size))
                left = right
                right = []
                aspect_term = []
                polarity = []
            aspect_term.append(sentence[i])
            polarity.append(polarity_tag[i])
        else:
            if doing_left:
                left.append(sentence[i])
            elif doing_right:
                right.append(sentence[i])
            else:
                doing_aspect = False
                doing_right = True
                right.append(sentence[i])
                
    ret_aspect_clusters.append(get_new_aspect_cluster(
        left, right, aspect_term, polarity, sentence, window_size=window_size))
    
    return ret_aspect_clusters

In [35]:
def upsample_polarity(all_aspect_clusters):
    neg_list = []
    neu_list = []
    pos_list = []

    for item in all_aspect_clusters:
        if item['polarity'] == 2:
            neg_list.append(item)
        elif item['polarity'] == 0:
            neu_list.append(item)
        else:
            pos_list.append(item)

    num_to_add = len(pos_list) - len(neg_list)
    for i in range(num_to_add):
        idx = random.randint(0, len(neg_list)-1)
        neg_list.append(neg_list[idx])

    num_to_add = len(pos_list) - len(neu_list)
    for i in range(num_to_add):
        idx = random.randint(0, len(neu_list)-1)
        neu_list.append(neu_list[idx])    

    all_aspect_clusters = neg_list
    all_aspect_clusters.extend(neu_list)
    all_aspect_clusters.extend(pos_list)  

    return all_aspect_clusters

In [36]:
# generate word_index list
def build_vocab(df):
    word_idx = {}
    for idx, word in enumerate(sorted(set(df.text.values))):
        word_idx[word] = idx + 1
    return word_idx

def gen_np_embedding(fn, word_idx, dim=100, emb=False):
    if emb:
        model = load_model(fn + ".bin")
    embedding = np.zeros((len(word_idx) + 2, dim))

    with open(fn, encoding="utf8") as f:
        for l in f:
            # for each line, get the word and its vector
            rec = l.rstrip().split(' ')
            if len(rec) == 2:  # skip the first line.
                continue
                # if the word in word_idx, fill the embedding
            if rec[0] in word_idx:
                embedding[word_idx[rec[0]]] = np.array([float(r) for r in rec[1:]])
    for w in word_idx:
        if embedding[word_idx[w]].sum() == 0.:
            if emb:
                embedding[word_idx[w]] = model.get_word_vector(w)
    return embedding

def create_train_data_restaurant(all_aspect_clusters,  word_idx, sent_len=83):

    train_X_local = np.zeros((len(all_aspect_clusters), sent_len), np.int16)
    train_X_global = np.zeros((len(all_aspect_clusters), sent_len), np.int16)

    train_y = np.zeros(len(all_aspect_clusters), np.int16)

    # iterate the asoect
    for sx, sent in enumerate(all_aspect_clusters):
        train_y[sx] = sent['polarity']
        global_sentence = sent['global_context']
        local_sentence = sent['local_context']
        try:
            for wx, word in enumerate(global_sentence):
                train_X_global[sx, wx] = word_idx[word]

            for wx, word in enumerate(local_sentence):
                train_X_local[sx, wx] = word_idx[word]

        except KeyError:
            continue

    return train_X_local, train_X_global, train_y


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    elif isinstance(data, dict):
        for k, v in data.items():
            data[k] = v.to(device)
        return data
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


def loss_fn(pred, label, num_label):
    pred = pred.view(-1,num_label)
    label = label.view(-1)
    loss = torch.nn.functional.cross_entropy(pred, label)
    return loss


def cal_acc(pred_tags, true_tags):
    if isinstance(pred_tags, list):
        pred_tags = torch.cat(pred_tags, 0)
        true_tags = torch.cat(true_tags, 0)
    acc = (pred_tags == true_tags).sum() / pred_tags.numel()
    f1 = f1_score(true_tags.cpu().numpy(), pred_tags.cpu().numpy(), labels=[0, 1, 2], average='weighted')
    cm = confusion_matrix(true_tags.cpu().numpy(), pred_tags.cpu().numpy())
    return acc, f1, cm


class Model(torch.nn.Module):
    def __init__(self, gen_emb, num_classes=3, alpha = 0.6):
        super(Model, self).__init__()
  
        self.embed = nn.Embedding.from_pretrained(torch.tensor(gen_emb, dtype=torch.float))

        self.lstm = nn.LSTM(gen_emb.shape[1], hidden_size=150, num_layers=1, bidirectional=True, batch_first=True)
        self.dense = nn.Linear(150*2, num_classes)

        self.alpha = alpha
        self.dropout = nn.Dropout(0.3)


    def forward(self, x_train_local, x_train_global):
        x_local = self.embed(x_train_local)
        
        local_seq_lengths = np.sum(np.array(x_train_local) !=0, axis=1)
   
        x_emb_l = torch.nn.utils.rnn.pack_padded_sequence(x_local, local_seq_lengths, batch_first=True, enforce_sorted=False)

        _, (h_n, _) = self.lstm(x_emb_l.float())

        h_n = torch.cat([h_n[-2,:,:], h_n[-1,:,:]], dim=1)

        x_global = self.embed(x_train_global)


        global_seq_lengths = np.sum(np.array(x_train_global) !=0, axis=1)

        x_emb_g = torch.nn.utils.rnn.pack_padded_sequence(x_global, global_seq_lengths, batch_first=True, enforce_sorted=False)

        _, (h_n_g,_) = self.lstm(x_emb_g.float())

        h_n_g = torch.cat([h_n_g[-2,:,:], h_n_g[-1,:,:]], dim=1)

        avg_pool = torch.div(torch.add(h_n * (1 - self.alpha), h_n_g * self.alpha), 2)
        out = self.dropout(avg_pool)

        out = self.dense(out)

        final_out = torch.nn.functional.softmax(out)

        return final_out, out

In [37]:
word_indx = build_vocab(df)
    
fn = DATA_DIR + 'glove.840B.300d.txt'
general_embedding = gen_np_embedding(fn, word_indx, dim=300, emb=False)

In [38]:
def get_classification_report(test_loader, model, model_path=None):
    if model_path is not None: # load the saved model
        print('Loading saved model from: {}'.format(model_path))
        model.load_state_dict(torch.load(model_path))
    model = to_device(model, device)   
    
    model.eval()
    final_pred_polarity_tags = []
    final_true_polarity_tags = []


    with torch.no_grad():
        for data in tqdm(test_loader, total=len(test_loader)):
            for i in range(len(data)):
                data[i] = data[i].to(device)

            l_feature, g_feature, label = data
            l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
            pred_logits, logits = model(l_feature, g_feature)

            pred_tags = pred_logits.max(-1)[1]

            final_pred_polarity_tags.extend(pred_tags)
            final_true_polarity_tags.extend(label)

    final_pred_polarity_tags = torch.stack(final_pred_polarity_tags).cpu()
    final_true_polarity_tags = torch.stack(final_true_polarity_tags).cpu()
        
    print(classification_report(final_true_polarity_tags, final_pred_polarity_tags, 
                                target_names=["Neutral", "Positive", "Negative"]))

## upsampling = False

In [41]:
lst = [0, 0.2, 0.4]
lst = [0.6, 0.8, 1]

for alpha_val in lst:
    choice = [2, 4, 8]
    upsampling = False

    for window in choice:

        print()
        print("*"*90)
        print("Alpha Size: {}".format(alpha_val))
        print("Window Size: {}".format(window))
        print("*"*90)
        print()

        all_aspect_clusters = []
        for i in range(len(sentences)):
            all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i], 
                                            window_size=window))

        local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

        X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)

        device = get_default_device()

        NUM_EPOCHS = 20
        TRAIN_BATCH_SIZE = 128
        VALID_BATCH_SIZE = 1024

        NUM_POLARITY_TAGS = 3

        history = {
            "train_loss": list(),
            "polarity_train_acc": list(),
            "valid_loss": list(),
            "polarity_valid_acc": list(),
        }

        best_loss = np.inf

        dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
        print(f"train samples:{len(dataset)}")
        train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

        dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
        print(f"valid samples:{len(dataset_valid)}")
        test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

        model = to_device(Model(general_embedding,  num_classes=3, alpha = alpha_val), device)

        torch.cuda.empty_cache()

        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = AdamW(parameters, lr=1e-4)


        for epoch in range(NUM_EPOCHS):
            train_losses = []
            train_acc = []
            test_loss = []
            test_acc = []
            train_f1 = []
            test_f1 = []

            model.train()
            preds = []
            masks = []
            labels = []
            for data in tqdm(train_loader, total=len(train_loader)):
                for i in range(len(data)):
                    data[i] = data[i].to(device)
                l_feature, g_feature, label = data
                l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()

                optimizer.zero_grad()
                pred_logits, logits = model(l_feature, g_feature)
                loss = loss_fn(logits, label, 3) #w/o softmax
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

                pred_tags = pred_logits.max(-1)[1] #with softmax
                preds.append(pred_tags)
                labels.append(label)

            avg_train_acc, avg_train_f1, train_cm = cal_acc(preds,labels)
            avg_train_loss = sum(train_losses) / len(train_losses)

            preds = []
            masks = []
            labels = []
            with torch.no_grad():
                for data in tqdm(test_loader, total=len(test_loader)):
                    for i in range(len(data)):
                        data[i] = data[i].to(device)
                    l_feature, g_feature, label = data
                    l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
                    pred_logits, logits = model(l_feature, g_feature)
                    loss = loss_fn(logits, label, 3) #w/o softmax

                    test_loss.append(loss.item())

                    pred_tags = pred_logits.max(-1)[1] #with softmax

                    preds.append(pred_tags)
                    labels.append(label)

            avg_test_acc, avg_test_f1, test_cm = cal_acc(preds,  labels)
            avg_test_loss = sum(test_loss) / len(test_loss)

            print(f"\nepoch {epoch}")
            print("\ttrain_loss:{:.3f} valid_loss:{:.3f}".format(avg_train_loss, avg_test_loss))
            print("\ttrain_acc:{:.2%} valid_acc:{:.2%}".format(avg_train_acc, avg_test_acc))
            print("\ttrain_f1:{:.3f} valid_f1:{:.3f}".format(avg_train_f1, avg_test_f1))
            print(f"\ttrain_confusion_matrix:\n{train_cm}")
            print(f"\tvalid_confusion_matrix:\n{test_cm}")

            if avg_test_loss < best_loss:
                torch.save(model.state_dict(), MODEL_PATH)
                best_loss = avg_test_loss    
                
            history['train_loss'].append(avg_train_loss)
            history['polarity_train_acc'].append(avg_train_acc.cpu().numpy())
            history['valid_loss'].append(avg_test_loss)
            history['polarity_valid_acc'].append(avg_test_acc.cpu().numpy())

        get_classification_report(test_loader, model, model_path=MODEL_PATH)
                
            



******************************************************************************************
Alpha Size: 0.6
Window Size: 2
******************************************************************************************

train samples:4641
valid samples:1161


  0%|          | 0/36 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  6.28it/s]



epoch 0
	train_loss:1.065 valid_loss:1.017
	train_acc:50.35% valid_acc:54.78%
	train_f1:0.370 valid_f1:0.388
	train_confusion_matrix:
[[  26  865   10]
 [  87 2267   41]
 [  61 1224   27]]
	valid_confusion_matrix:
[[  0 211   0]
 [  0 636   0]
 [  0 314   0]]


  0%|          | 0/36 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  5.90it/s]



epoch 1
	train_loss:1.005 valid_loss:0.950
	train_acc:52.00% valid_acc:54.78%
	train_f1:0.358 valid_f1:0.388
	train_confusion_matrix:
[[   0  903    0]
 [   1 2387    1]
 [   4 1303    9]]
	valid_confusion_matrix:
[[  0 211   0]
 [  0 636   0]
 [  0 314   0]]


...


epoch 18
	train_loss:0.635 valid_loss:0.706
	train_acc:73.91% valid_acc:70.63%
	train_f1:0.717 valid_f1:0.697
	train_confusion_matrix:
[[ 237  320  333]
 [  84 2127  203]
 [  78  184 1042]]
	valid_confusion_matrix:
[[ 80  69  72]
 [ 46 516  50]
 [ 33  71 224]]


  0%|          | 0/36 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  5.72it/s]



epoch 19
	train_loss:0.626 valid_loss:0.713
	train_acc:74.26% valid_acc:71.15%
	train_f1:0.724 valid_f1:0.694
	train_confusion_matrix:
[[ 266  316  313]
 [  96 2136  180]
 [ 102  179 1020]]
	valid_confusion_matrix:
[[ 64  73  84]
 [ 26 523  63]
 [ 17  72 239]]
Loading saved model from: model_task2.bin


  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  5.83it/s]

              precision    recall  f1-score   support

     Neutral       0.51      0.37      0.43       221
    Positive       0.78      0.84      0.81       612
    Negative       0.65      0.68      0.67       328

    accuracy                           0.71      1161
   macro avg       0.65      0.63      0.64      1161
weighted avg       0.69      0.71      0.70      1161



## upsampling = True

In [42]:
import random
lst = [0, 0.2, 0.4]
#[0.6, 0.8, 1]

for alpha_val in lst:
    choice = [2, 4, 8]
    upsampling = True

    for window in choice:

        print()
        print("*"*90)
        print("Alpha Size: {}".format(alpha_val))
        print("Window Size: {}".format(window))
        print("*"*90)
        print()

        all_aspect_clusters = []
        for i in range(len(sentences)):
            all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i], 
                                            window_size=window))
        #if upsampling:
        #    all_aspect_clusters = upsample_polarity(all_aspect_clusters)

        #local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

        #X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)

        train_aspect_clusters, test_aspect_clusters = train_test_split(all_aspect_clusters, random_state=42, test_size=VALID_SIZE)
        
        if upsampling:
            train_aspect_clusters = upsample_polarity(train_aspect_clusters)

        X_l_train, X_g_train, y_train = create_train_data_restaurant(train_aspect_clusters ,word_indx, sent_len=85)

        X_l_valid, X_g_valid, y_valid = create_train_data_restaurant(test_aspect_clusters ,word_indx, sent_len=85)

        device = get_default_device()

        NUM_EPOCHS = 20
        TRAIN_BATCH_SIZE = 128
        VALID_BATCH_SIZE = 1024

        NUM_POLARITY_TAGS = 3

        history = {
            "train_loss": list(),
            "polarity_train_acc": list(),
            "valid_loss": list(),
            "polarity_valid_acc": list(),
        }

        best_loss = np.inf

        dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
        print(f"train samples:{len(dataset)}")
        train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

        dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
        print(f"valid samples:{len(dataset_valid)}")
        test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

        model = to_device(Model(general_embedding,  num_classes=3, alpha = alpha_val), device)

        torch.cuda.empty_cache()

        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = AdamW(parameters, lr=1e-4)


        for epoch in range(NUM_EPOCHS):
            train_losses = []
            train_acc = []
            test_loss = []
            test_acc = []
            train_f1 = []
            test_f1 = []

            model.train()
            preds = []
            masks = []
            labels = []
            for data in tqdm(train_loader, total=len(train_loader)):
                for i in range(len(data)):
                    data[i] = data[i].to(device)
                l_feature, g_feature, label = data
                l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()

                optimizer.zero_grad()
                pred_logits, logits = model(l_feature, g_feature)
                loss = loss_fn(logits, label, 3) #w/o softmax
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

                pred_tags = pred_logits.max(-1)[1] #with softmax
                preds.append(pred_tags)
                labels.append(label)

            avg_train_acc, avg_train_f1, train_cm = cal_acc(preds,labels)
            avg_train_loss = sum(train_losses) / len(train_losses)

            preds = []
            masks = []
            labels = []
            with torch.no_grad():
                for data in tqdm(test_loader, total=len(test_loader)):
                    for i in range(len(data)):
                        data[i] = data[i].to(device)
                    l_feature, g_feature, label = data
                    l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
                    pred_logits, logits = model(l_feature, g_feature)
                    loss = loss_fn(logits, label, 3) #w/o softmax

                    test_loss.append(loss.item())

                    pred_tags = pred_logits.max(-1)[1] #with softmax

                    preds.append(pred_tags)
                    labels.append(label)

            avg_test_acc, avg_test_f1, test_cm = cal_acc(preds,  labels)
            avg_test_loss = sum(test_loss) / len(test_loss)

            print(f"\nepoch {epoch}")
            print("\ttrain_loss:{:.3f} valid_loss:{:.3f}".format(avg_train_loss, avg_test_loss))
            print("\ttrain_acc:{:.2%} valid_acc:{:.2%}".format(avg_train_acc, avg_test_acc))
            print("\ttrain_f1:{:.3f} valid_f1:{:.3f}".format(avg_train_f1, avg_test_f1))
            print(f"\ttrain_confusion_matrix:\n{train_cm}")
            print(f"\tvalid_confusion_matrix:\n{test_cm}")

            if avg_test_loss < best_loss:
                torch.save(model.state_dict(), MODEL_PATH)
                best_loss = avg_test_loss    
                
            history['train_loss'].append(avg_train_loss)
            history['polarity_train_acc'].append(avg_train_acc.cpu().numpy())
            history['valid_loss'].append(avg_test_loss)
            history['polarity_valid_acc'].append(avg_test_acc.cpu().numpy())

        get_classification_report(test_loader, model, model_path=MODEL_PATH)
                
            



******************************************************************************************
Alpha Size: 0
Window Size: 2
******************************************************************************************

train samples:7311
valid samples:1161


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  5.92it/s]



epoch 0
	train_loss:1.086 valid_loss:1.065
	train_acc:43.50% valid_acc:57.88%
	train_f1:0.385 valid_f1:0.578
	train_confusion_matrix:
[[ 780 1515  135]
 [ 293 2060   81]
 [ 558 1540  334]]
	valid_confusion_matrix:
[[ 75  73  80]
 [ 77 428 102]
 [ 57 100 169]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  6.37it/s]



epoch 1
	train_loss:1.051 valid_loss:1.005
	train_acc:55.44% valid_acc:58.48%
	train_f1:0.543 valid_f1:0.581
	train_confusion_matrix:
[[ 846  714  874]
 [ 239 1667  525]
 [ 319  580 1532]]
	valid_confusion_matrix:
[[ 58  67 103]
 [ 52 403 152]
 [ 35  73 218]]


...


epoch 18
	train_loss:0.533 valid_loss:0.746
	train_acc:79.88% valid_acc:70.20%
	train_f1:0.799 valid_f1:0.711
	train_confusion_matrix:
[[1882  203  346]
 [ 319 1944  169]
 [ 282  149 2002]]
	valid_confusion_matrix:
[[130  36  62]
 [ 77 450  80]
 [ 67  24 235]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  5.73it/s]



epoch 19
	train_loss:0.517 valid_loss:0.779
	train_acc:80.63% valid_acc:69.77%
	train_f1:0.807 valid_f1:0.705
	train_confusion_matrix:
[[1912  192  326]
 [ 300 1960  173]
 [ 274  148 2011]]
	valid_confusion_matrix:
[[119  31  78]
 [ 74 441  92]
 [ 54  22 250]]
Loading saved model from: model_task2.bin


  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  5.69it/s]

              precision    recall  f1-score   support

     Neutral       0.46      0.57      0.51       228
    Positive       0.86      0.76      0.80       607
    Negative       0.64      0.68      0.66       326

    accuracy                           0.70      1161
   macro avg       0.65      0.67      0.66      1161
weighted avg       0.72      0.70      0.71      1161



In [43]:
import random
lst = [0.6, 0.8, 1]

for alpha_val in lst:
    choice = [2, 4, 8]
    upsampling = True

    for window in choice:

        print()
        print("*"*90)
        print("Alpha Size: {}".format(alpha_val))
        print("Window Size: {}".format(window))
        print("*"*90)
        print()

        all_aspect_clusters = []
        for i in range(len(sentences)):
            all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i], 
                                            window_size=window))
        #if upsampling:
        #    all_aspect_clusters = upsample_polarity(all_aspect_clusters)

        #local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

        #X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)

        train_aspect_clusters, test_aspect_clusters = train_test_split(all_aspect_clusters, random_state=42, test_size=VALID_SIZE)
        
        if upsampling:
            train_aspect_clusters = upsample_polarity(train_aspect_clusters)

        X_l_train, X_g_train, y_train = create_train_data_restaurant(train_aspect_clusters ,word_indx, sent_len=85)

        X_l_valid, X_g_valid, y_valid = create_train_data_restaurant(test_aspect_clusters ,word_indx, sent_len=85)

        device = get_default_device()

        NUM_EPOCHS = 20
        TRAIN_BATCH_SIZE = 128
        VALID_BATCH_SIZE = 1024

        NUM_POLARITY_TAGS = 3

        history = {
            "train_loss": list(),
            "polarity_train_acc": list(),
            "valid_loss": list(),
            "polarity_valid_acc": list(),
        }

        best_loss = np.inf

        dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
        print(f"train samples:{len(dataset)}")
        train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

        dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
        print(f"valid samples:{len(dataset_valid)}")
        test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

        model = to_device(Model(general_embedding,  num_classes=3, alpha = alpha_val), device)

        torch.cuda.empty_cache()

        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = AdamW(parameters, lr=1e-4)


        for epoch in range(NUM_EPOCHS):
            train_losses = []
            train_acc = []
            test_loss = []
            test_acc = []
            train_f1 = []
            test_f1 = []

            model.train()
            preds = []
            masks = []
            labels = []
            for data in tqdm(train_loader, total=len(train_loader)):
                for i in range(len(data)):
                    data[i] = data[i].to(device)
                l_feature, g_feature, label = data
                l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()

                optimizer.zero_grad()
                pred_logits, logits = model(l_feature, g_feature)
                loss = loss_fn(logits, label, 3) #w/o softmax
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

                pred_tags = pred_logits.max(-1)[1] #with softmax
                preds.append(pred_tags)
                labels.append(label)

            avg_train_acc, avg_train_f1, train_cm = cal_acc(preds,labels)
            avg_train_loss = sum(train_losses) / len(train_losses)

            preds = []
            masks = []
            labels = []
            with torch.no_grad():
                for data in tqdm(test_loader, total=len(test_loader)):
                    for i in range(len(data)):
                        data[i] = data[i].to(device)
                    l_feature, g_feature, label = data
                    l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
                    pred_logits, logits = model(l_feature, g_feature)
                    loss = loss_fn(logits, label, 3) #w/o softmax

                    test_loss.append(loss.item())

                    pred_tags = pred_logits.max(-1)[1] #with softmax

                    preds.append(pred_tags)
                    labels.append(label)

            avg_test_acc, avg_test_f1, test_cm = cal_acc(preds,  labels)
            avg_test_loss = sum(test_loss) / len(test_loss)

            print(f"\nepoch {epoch}")
            print("\ttrain_loss:{:.3f} valid_loss:{:.3f}".format(avg_train_loss, avg_test_loss))
            print("\ttrain_acc:{:.2%} valid_acc:{:.2%}".format(avg_train_acc, avg_test_acc))
            print("\ttrain_f1:{:.3f} valid_f1:{:.3f}".format(avg_train_f1, avg_test_f1))
            print(f"\ttrain_confusion_matrix:\n{train_cm}")
            print(f"\tvalid_confusion_matrix:\n{test_cm}")

            if avg_test_loss < best_loss:
                torch.save(model.state_dict(), MODEL_PATH)
                best_loss = avg_test_loss    
                
            history['train_loss'].append(avg_train_loss)
            history['polarity_train_acc'].append(avg_train_acc.cpu().numpy())
            history['valid_loss'].append(avg_test_loss)
            history['polarity_valid_acc'].append(avg_test_acc.cpu().numpy())

        get_classification_report(test_loader, model, model_path=MODEL_PATH)
                
            



******************************************************************************************
Alpha Size: 0.6
Window Size: 2
******************************************************************************************

train samples:7311
valid samples:1161


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  6.45it/s]



epoch 0
	train_loss:1.085 valid_loss:1.068
	train_acc:45.13% valid_acc:58.14%
	train_f1:0.439 valid_f1:0.587
	train_confusion_matrix:
[[ 698  365 1368]
 [ 420  923 1091]
 [ 460  299 1672]]
	valid_confusion_matrix:
[[ 81  51  96]
 [ 84 399 124]
 [ 56  75 195]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  6.43it/s]



epoch 1
	train_loss:1.043 valid_loss:0.974
	train_acc:54.59% valid_acc:60.12%
	train_f1:0.536 valid_f1:0.604
	train_confusion_matrix:
[[ 845  687  900]
 [ 269 1689  476]
 [ 417  564 1449]]
	valid_confusion_matrix:
[[ 73  52 103]
 [ 70 419 118]
 [ 54  66 206]]


...


epoch 18
	train_loss:0.563 valid_loss:0.789
	train_acc:78.08% valid_acc:69.94%
	train_f1:0.781 valid_f1:0.706
	train_confusion_matrix:
[[1764  259  408]
 [ 296 1949  188]
 [ 268  180 1984]]
	valid_confusion_matrix:
[[117  36  75]
 [ 74 449  84]
 [ 54  26 246]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.68it/s]



epoch 19
	train_loss:0.555 valid_loss:0.796
	train_acc:78.69% valid_acc:68.99%
	train_f1:0.787 valid_f1:0.698
	train_confusion_matrix:
[[1799  251  383]
 [ 282 1936  211]
 [ 266  162 2006]]
	valid_confusion_matrix:
[[121  37  70]
 [ 90 439  78]
 [ 55  30 241]]
Loading saved model from: model_task2.bin


  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_10911/1044936714.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_out = torch.nn.functional.softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.41it/s]

              precision    recall  f1-score   support

     Neutral       0.50      0.46      0.48       228
    Positive       0.84      0.79      0.81       607
    Negative       0.62      0.72      0.67       326

    accuracy                           0.71      1161
   macro avg       0.65      0.66      0.65      1161
weighted avg       0.71      0.71      0.71      1161



In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['polarity_train_acc'], label='train accuracy')
plt.plot(history['polarity_valid_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0.4, 0.9])